# Generate dollar bars
courtesy of towards data science

In [1]:
import pandas as pd
import numpy as np
from IPython.display import clear_output


In [2]:
df = pd.read_csv("../data/BTCUSDT-trades-55.csv")

In [3]:
df

,a,p,q,f,l,T,m,M
0,275000000,7360.10,0.003123,299599745,299599745,1587651051862,False,True
1,275000001,7360.10,0.002579,299599746,299599746,1587651051887,False,True
2,275000002,7360.00,0.005852,299599747,299599747,1587651051898,True,True
3,275000003,7360.14,0.003530,299599748,299599748,1587651051960,False,True
4,275000004,7360.14,0.002036,299599749,299599749,1587651051971,False,True
...,...,...,...,...,...,...,...,...
4999995,279999995,9171.35,0.049100,304995279,304995279,1588236463282,False,True
4999996,279999996,9170.82,0.063568,304995280,304995280,1588236463451,False,True
4999997,279999997,9170.51,0.002181,304995281,304995281,1588236463462,True,True
4999998,279999998,9170.81,0.036487,304995282,304995282,1588236463526,False,True


In [4]:
df_np = df[["T", "p", "q"]].to_numpy()

In [36]:
# expects a numpy array with trades
# each trade is composed of: [time, price, quantity]
def generate_dollarbars(trades, frequency=100000):
    times = trades[:,0]
    prices = trades[:,1]
    volumes = trades[:,2]
    ans = np.zeros(shape=(len(prices), 6))
    candle_counter = 0
    dollars = 0
    lasti = 0
    carry = {"time":None, "price":0, "volume":0}
    for i in range(prices.shape[0]):
        
        while carry["price"]*carry["volume"]>=frequency:
            ans[candle_counter][0] = carry["time"]                         # time
            ans[candle_counter][1] = carry["price"]                    # open
            ans[candle_counter][2] = carry["price"]         # high
            ans[candle_counter][3] = carry["price"]         # low
            ans[candle_counter][4] = carry["price"]                        # close
            ans[candle_counter][5] = carry["volume"]  # volume
            candle_counter += 1
            lasti = i+1
            carry["volume"] = carry["volume"] - frequency/carry["price"]
            
        dollars += carry["volume"]*carry["price"] + volumes[i]*prices[i]
        if dollars >= frequency:
            carry = {"time":times[i], "price":prices[i], "volume":(dollars-frequency)/prices[i]}
            ans[candle_counter][0] = times[i]                          # time
            ans[candle_counter][1] = prices[lasti]                     # open
            ans[candle_counter][2] = np.max(prices[lasti:i+1])         # high
            ans[candle_counter][3] = np.min(prices[lasti:i+1])         # low
            ans[candle_counter][4] = prices[i]                         # close
            ans[candle_counter][5] = np.sum(volumes[lasti:i+1])  # volume
            candle_counter += 1
            lasti = i+1
            dollars = 0
            clear_output()
            print(f"currently on index {i} out of {prices.shape[0]}")
    return ans[:candle_counter]

In [38]:
# expects a numpy array with trades
# each trade is composed of: [time, price, quantity]
def generate_dollarbars(trades, frequency=100000):
    times = trades[:,0]
    prices = trades[:,1]
    volumes = trades[:,2]
    ans = np.zeros(shape=(len(prices), 6))
    candle_counter = 0
    dollars = 0
    lasti = 0
    carry = 0
    i = 0
    while i < prices.shape[0]:
        dollars += volumes[i]*prices[i]
        if dollars >= frequency:
            carry = (dollars-frequency)/prices[i]
            times = np.insert(times, i+1, times[i])
            prices = np.insert(prices, i+1, prices[i])
            volumes = np.insert(volumes, i+1, carry)
            ans[candle_counter][0] = times[i]                          # time
            ans[candle_counter][1] = prices[lasti]                     # open
            ans[candle_counter][2] = np.max(prices[lasti:i+1])         # high
            ans[candle_counter][3] = np.min(prices[lasti:i+1])         # low
            ans[candle_counter][4] = prices[i]                         # close
            ans[candle_counter][5] = np.sum(volumes[lasti:i+1])-carry  # volume
            candle_counter += 1
            lasti = i+1
            dollars = 0
            clear_output()
            print(f"currently on index {i} out of {prices.shape[0]}")
        i += 1
    return ans[:candle_counter]

In [7]:
a = generate_dollarbars(df_np)

In [8]:
a

array([[1.58765106e+12, 7.36010000e+03, 7.36071000e+03, 7.36000000e+03,
        7.36070000e+03, 1.52300570e+01],
       [1.58765106e+12, 7.36071000e+03, 7.36170000e+03, 7.36000000e+03,
        7.36127000e+03, 1.50770270e+01],
       [1.58765106e+12, 7.36169000e+03, 7.36246000e+03, 7.35900000e+03,
        7.35900000e+03, 1.39237000e+01],
       ...,
       [1.58823641e+12, 9.17426000e+03, 9.17426000e+03, 9.17050000e+03,
        9.17400000e+03, 1.15850020e+01],
       [1.58823645e+12, 9.17401000e+03, 9.17401000e+03, 9.17200000e+03,
        9.17293000e+03, 1.10119350e+01],
       [1.58823646e+12, 9.17293000e+03, 9.17394000e+03, 9.17051000e+03,
        9.17143000e+03, 1.14769000e+01]])

In [9]:
b = pd.DataFrame(a, columns=["time", "open", "high", "low", "close", "volume"])

In [10]:
b

,time,open,high,low,close,volume
0,1.587651e+12,7360.10,7360.71,7360.00,7360.70,15.230057
1,1.587651e+12,7360.71,7361.70,7360.00,7361.27,15.077027
2,1.587651e+12,7361.69,7362.46,7359.00,7359.00,13.923700
3,1.587651e+12,7358.77,7360.99,7358.23,7358.59,13.609477
4,1.587651e+12,7358.58,7359.70,7355.00,7355.13,13.674025
...,...,...,...,...,...,...
44179,1.588236e+12,9172.23,9174.88,9170.00,9174.82,11.073791
44180,1.588236e+12,9174.84,9175.48,9173.61,9174.27,10.962070
44181,1.588236e+12,9174.26,9174.26,9170.50,9174.00,11.585002
44182,1.588236e+12,9174.01,9174.01,9172.00,9172.93,11.011935


In [11]:
b.to_csv("../data/testing.csv", index=False)

In [5]:
# Towards Data Science base
# expects a numpy array with trades
# each trade is composed of: [time, price, quantity]
def generate_dollarbars(trades, frequency=100000):
    times = trades[:,0]
    prices = trades[:,1]
    volumes = trades[:,2]
    ans = np.zeros(shape=(len(prices), 6))
    candle_counter = 0
    dollars = 0
    lasti = 0
    for i in range(len(prices)):
        dollars += volumes[i]*prices[i]
        if dollars >= frequency:
            ans[candle_counter][0] = times[i]                          # time
            ans[candle_counter][1] = prices[lasti]                     # open
            ans[candle_counter][2] = np.max(prices[lasti:i+1])         # high
            ans[candle_counter][3] = np.min(prices[lasti:i+1])         # low
            ans[candle_counter][4] = prices[i]                         # close
            ans[candle_counter][5] = np.sum(volumes[lasti:i+1])        # volume
            candle_counter += 1
            lasti = i+1
            dollars = 0
#             clear_output()
#             print(f"currently on index {i} out of {prices.shape[0]}")
    return ans[:candle_counter]